In [ ]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

Import and initiate findspark to begin with.
And import pyspark

Now start SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movie Lens Data Case Study").getOrCreate()

Create DataFrame from data source - csv file

In [ ]:
movieDF = spark.read.load("u.data", format="csv", sep="\t", inferSchema="true")

Different types of operations on DataFrames

In [ ]:
movieDF.printSchema()

Rename the columns with meaningful names for easy reference. 
Import the required libraries for this purpose

In [ ]:
from pyspark.sql.functions import *

In [ ]:
movieDF2 = movieDF.select(col('_c0').alias('userId'),col('_c1').alias('movieId'),col('_c2').alias('rating'),col('_c3').alias('timeStamp'))

In [ ]:
movieDF2.printSchema()

Similarly read and load user data file into a dataframe and rename the columns

In [ ]:
userDF = spark.read.load("u.user", format="csv", sep="|", inferSchema="true")
userDF.printSchema()

In [ ]:
userDF2 = userDF.withColumnRenamed('_c0', 'userId').withColumnRenamed('_c1', 'age').withColumnRenamed('_c2', 'gender').withColumnRenamed('_c3', 'occupation').withColumnRenamed('_c4', 'zipCode')

In [ ]:
userDF2.printSchema()

1.	Give gender-wise breakup of the users

In [ ]:
userDF2.groupBy('gender').count().show()

Create temp views for user dataframe and movie dataframe

In [ ]:
movieDF2.createOrReplaceTempView("movies")
userDF2.createOrReplaceTempView("users")

In [ ]:
spark.sql("SELECT gender , count(*) from users group by gender").show()

2.	Give the top 5 movies which are reviewed maximum number of times

In [ ]:
spark.sql("SELECT movieId , count(*) as moviecount from movies group by movieId order by moviecount desc limit 5").show()

3.	Give the top 5 users who reviewed maximum number of movies

In [ ]:
spark.sql("SELECT userId , count(*) as usercount from movies group by userId order by usercount desc limit 5").show()

4.	List the top 10 movies which received highest number of 5 star ratings

In [ ]:
spark.sql("SELECT movieId, rating, count(*) as highratedmoviecount from movies group by movieId, rating order by rating desc, highratedmoviecount desc limit 10").show()

5.	List the top 10 users who gave highest number of 5 star ratings

In [ ]:
spark.sql("SELECT userId, rating, count(*) as maxusercount from movies group by userId, rating order by rating desc, maxusercount desc limit 10").show()

6.	Prepare a dataframe from user data that has the records transformed

In [ ]:
userDF3 = spark.sql("SELECT userId, int(round(age/10,0)) as ageGroup, gender, CASE WHEN gender='F' THEN 1 else 0 END as female , CASE WHEN gender='M' THEN 1 else 0 END as male, occupation, zipCode from users")

In [ ]:
userDF3.show()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="occupation", outputCol="occupationId")

In [ ]:
userDF4 = indexer.fit(userDF3).transform(userDF3)

In [ ]:
userDF4.printSchema()

In [ ]:
userDF4.show()